In [12]:
#VERSION 3.8.10

import cv2
import numpy as np
import scipy
import matplotlib.pylab as plt
from skimage import io


img = cv2.imread("./images/final_exam/Templates/Template-1.png", 0)  
cap = cv2.VideoCapture("./videos/final_exam/Dataset-1/left_output-1.avi", 0)



# Features


sift = cv2.xfeatures2d.SIFT_create()
kp_image, desc_image = sift.detectAndCompute(img, None)
fl = cv2.flann
bf = cv2.BFMatcher


# matching
index_params = dict(algorithm=0, trees=5)
search_params = dict()
bf = cv2.FlannBasedMatcher(index_params, search_params)


while cap.isOpened() :


    ret, frame = cap.read()
    grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  
    kp_grayframe, desc_grayframe = sift.detectAndCompute(grayframe, None)
    matches = bf.knnMatch(desc_image, desc_grayframe, k=2)
    good_points = []
   


    for m, n in matches:
        if m.distance < 0.79999 * n.distance:
            good_points.append(m)


            
    # img3 = cv2.drawMatches(img, kp_image, grayframe, kp_grayframe, good_points, grayframe)



    # Homo


    if len(good_points) > 10:


        query_pts = np.float32([kp_image[m.queryIdx].pt for m in good_points]).reshape(-1, 1, 2)
        train_pts = np.float32([kp_grayframe[m.trainIdx].pt for m in good_points]).reshape(-1, 1, 2)
        matrix, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC, 0.5)
        matches_mask = mask.ravel().tolist()


        # transform


        h, w = img.shape
        point = np.float32([[0, 0], [0, h], [w, h], [w, 0]]).reshape(-1, 1, 2)
        dest = cv2.perspectiveTransform(point, matrix)
        LASTDETECT = cv2.polylines(frame, [np.int32(dest)], True, (255, 0, 0), 3)
        cv2.imshow("Homography", LASTDETECT)

        
    else:
        cv2.imshow("Homography", grayframe)
        # cv2.imshow("Image", img)
        # cv2.imshow("grayFrame", grayframe)
        # cv2.imshow("img3", img3)
    key = cv2.waitKey(35) & 0xFF
    if key == 27 or key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

################################################################################################

#หลังจากนี้กระผมทำไม่ได้เเล้วขอรับ

################################################################################################
params_dir = os.getcwd()+'./new-camera_params'
print(params_dir)

#load camera parameters
K = np.load(params_dir+'K.npy')
dist = np.load(params_dir+'dist.npy')

print("Camera matrix")
print(K)
print("Len distortion")
print(dist)
def augmented_image(frame,im_src, pts_src, pts_dst):
    
    # Calculate Homography
    h, status = cv2.findHomography(pts_src, pts_dst)

    # Warp source image to destination based on homography
    warped_image = cv2.warpPerspective(im_src, h, (frame.shape[1],frame.shape[0]))
            
    # Prepare a mask representing region to copy from the warped image into the original frame.
    mask = np.zeros([frame.shape[0], frame.shape[1]], dtype=np.uint8)
    cv2.fillConvexPoly(mask, np.int32(pts_dst), (255, 255, 255), cv2.LINE_AA)
    im_out = cv2.add(frame, warped_image, mask=cv2.bitwise_not(mask))
    im_out = cv2.add(im_out, warped_image)
    
    return im_out
def write_text(img, pose, dy, text) :
    x0 = pose[0]
    y0 = pose[1]
    for i, line in enumerate(text.split('\n')) :
        y = y0 + i*dy
        cv2.putText(img, line, np.int32([x0, y]), cv2.FONT_HERSHEY_COMPLEX, 0.75, (50,200,255), 2)
        markerCorners, markerIds, rejectedCandidates = cv2.aruco.detectMarkers(frame, AruCo_dict, parameters = AruCo_params)
        
        if len(markerCorners) > 0:
            img = cv2.aruco.drawDetectedMarkers(frame, markerCorners)
            rvecs, tvecs, points = cv2.aruco.estimatePoseSingleMarkers(markerCorners , 0.05, K, dist)
            for (rvec, tvec, id, corner) in zip(rvecs, tvecs, markerIds, markerCorners) :
                img = cv2.aruco.drawAxis(frame, K, dist, rvec, tvec, 0.05)
                x = tvec[0,0]
                y = tvec[0,1]
                z = tvec[0,2]
                text = "id: {}\n pose:\n {:.3f}\n {:.3f}\n {:.3f}".format(id, x, y, z)
                cX = (corner[0,0][0] + corner[0,2][0]) / 2
                cY = (corner[0,0][1] + corner[0,2][1]) / 2
                write_text(img, (cX, cY), 20, text)

    

c:\Users\BEE\Downloads\010723305-main./new-camera_params


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\BEE\\Downloads\\010723305-main./new-camera_paramsK.npy'

In [ ]:
# params_dir = os.getcwd()+'/camera_params/monocular_camera_params/'
# print(params_dir)

# #load camera parameters
# K = np.load(params_dir+'K.npy')
# dist = np.load(params_dir+'dist.npy')

# print("Camera matrix")
# print(K)
# print("Len distortion")
# print(dist)


In [ ]:
# def augmented_image(frame,im_src, pts_src, pts_dst):
    
#     # Calculate Homography
#     h, status = cv2.findHomography(pts_src, pts_dst)

#     # Warp source image to destination based on homography
#     warped_image = cv2.warpPerspective(im_src, h, (frame.shape[1],frame.shape[0]))
            
#     # Prepare a mask representing region to copy from the warped image into the original frame.
#     mask = np.zeros([frame.shape[0], frame.shape[1]], dtype=np.uint8)
#     cv2.fillConvexPoly(mask, np.int32(pts_dst), (255, 255, 255), cv2.LINE_AA)
#     im_out = cv2.add(frame, warped_image, mask=cv2.bitwise_not(mask))
#     im_out = cv2.add(im_out, warped_image)
    
#     return im_out

In [ ]:
# def write_text(img, pose, dy, text) :
#     x0 = pose[0]
#     y0 = pose[1]
#     for i, line in enumerate(text.split('\n')) :
#         y = y0 + i*dy
#         cv2.putText(img, line, np.int32([x0, y]), cv2.FONT_HERSHEY_COMPLEX, 0.75, (50,200,255), 2)

In [ ]:
#  markerCorners, markerIds, rejectedCandidates = cv2.aruco.detectMarkers(frame, AruCo_dict, parameters = AruCo_params)
        
#         if len(markerCorners) > 0:
#             img = cv2.aruco.drawDetectedMarkers(frame, markerCorners)
#             rvecs, tvecs, points = cv2.aruco.estimatePoseSingleMarkers(markerCorners , 0.05, K, dist)
#             for (rvec, tvec, id, corner) in zip(rvecs, tvecs, markerIds, markerCorners) :
#                 img = cv2.aruco.drawAxis(frame, K, dist, rvec, tvec, 0.05)
#                 x = tvec[0,0]
#                 y = tvec[0,1]
#                 z = tvec[0,2]
#                 text = "id: {}\n pose:\n {:.3f}\n {:.3f}\n {:.3f}".format(id, x, y, z)
#                 cX = (corner[0,0][0] + corner[0,2][0]) / 2
#                 cY = (corner[0,0][1] + corner[0,2][1]) / 2
#                 write_text(img, (cX, cY), 20, text)